In [74]:
import tensorflow as tf
import _pickle as cPickle
import numpy as np
from random import shuffle
import matplotlib
import matplotlib.pyplot as plt

# chuyển lại default interpolation về 'bilinear' như matplotlib 1.4.0, thay vì 'nearest' như 2.0.0
# chế độ 'nearest' tự động resample, nên ta cũng phải chỉnh lại thông số này
# https://matplotlib.org/users/dflt_style_changes.html#interpolation 
matplotlib.rcParams['image.interpolation'] = 'bilinear'
matplotlib.rcParams['image.resample'] = False

In [75]:
# neuronsX = number of neurons in layer X

# convolution
neurons1 = 32
neurons2 = 64
neurons3 = 128
neurons4 = 256

# FC
fc_neurons1 = 2**9
fc_neurons2 = 2**6

# others
LEARNING_RATE = 1e-4
KEEP_PROBABILITY = 0.85

So 3 rows per image (for R G B); flatten (32x32 = 1024)

In [76]:
def unpickle(filename):
    fo = open(filename, 'rb')
    fdict = cPickle.load(fo, encoding='latin1')
    fo.close()
    return fdict
def flatten_image(image):
    return [item for sublist in image for item in sublist]

def format_data(data):
    images = []
    data_size = int(len(data)/3)
    for i in range(data_size):
        images.append(flatten_image(data[3*i:3*i+3]))
    return images

def format_label(labels):
    prep = [np.array([0]*9) for i in range(len(labels))]
    for i in range(len(labels)):
        prep[i][labels[i]] += 1
    return prep

def get_batch(filename):
    data_batch = unpickle(filename)
    data = format_data(data_batch['data'])
    labels = format_label(data_batch['labels'])
    return data, labels

def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

### Layer 1 (Convolutional)

In [77]:
x = tf.placeholder(tf.float32, [None, 3*32*32])
x_image = tf.reshape(x, [-1,32,32,3])
y_ = tf.placeholder(tf.float32, [None, 9])

In [78]:
W_conv1 = weight_variable([2, 2, 3, neurons1])
b_conv1 = bias_variable([neurons1])

h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

### Layer 2 (Convolutional)

In [79]:
W_conv2 = weight_variable([2, 2, neurons1, neurons2])
b_conv2 = bias_variable([neurons2])

h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)

### Layer 3 (Convolutional)

In [80]:
W_conv3 = weight_variable([2, 2, neurons2, neurons3])
b_conv3 = bias_variable([neurons3])

h_conv3 = tf.nn.relu(conv2d(h_pool2, W_conv3) + b_conv3)
h_pool3 = max_pool_2x2(h_conv3)

### Layer 4 (Convolutional)


In [81]:
W_conv4 = weight_variable([2, 2, neurons3, neurons4])
b_conv4 = bias_variable([neurons4])

h_conv4 = tf.nn.relu(conv2d(h_pool3, W_conv4) + b_conv4)
h_pool4 = max_pool_2x2(h_conv4)

### FC Layer 1

In [82]:
W_fc1 = weight_variable([2 * 2 * neurons4, fc_neurons1])
b_fc1 = bias_variable([neurons4])

h_pool4_flat = tf.reshape(h_pool4, [-1, 2*2*neurons4])
h_fc1 = tf.nn.relu(tf.matmul(h_pool4_flat, W_fc1) + b_fc1)

### FC Layer 2

In [83]:
W_fc2 = weight_variable([fc_neurons1, fc_neurons2])
b_fc2 = bias_variable([fc_neurons2])

h_fc2 = tf.nn.relu(tf.matmul(h_fc1, W_fc2) + b_fc2)

### Final layer (softmax, with dropout)

In [84]:
keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)
h_fc2_drop = tf.nn.dropout(h_fc2, keep_prob)

In [85]:
W_fc3 = weight_variable([fc_neurons2, 9])
b_fc3 = bias_variable([9])

y_conv = tf.matmul(h_fc2_drop, W_fc3) + b_fc3

### Training

In [86]:
sess = tf.InteractiveSession()
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(y_conv, y_))
train_step = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
sess.run(tf.global_variables_initializer())

In [87]:
def train_cycle():
    train_order = list(range(1, 6))
    shuffle(train_order)
    for i in train_order:
        batch_xs, batch_ys = get_batch('cucumber_data/p1/data_batch_{}'.format(i))
        train_step.run(feed_dict={x: batch_xs, y_: batch_ys, keep_prob: KEEP_PROBABILITY})
    
def test_cycle():
    correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    test_x, test_y = get_batch('cucumber_data/p1/test_batch')
    print(sess.run(accuracy, feed_dict={x: test_x,
                                      y_: test_y,
                                      keep_prob: 1.0}))

In [88]:
for i in range(600):
    train_cycle()
    if i % 20 == 0:
        print("Round {}".format(i))
        test_cycle()

Round 0
0.0969697
Round 10
0.276768
Round 20
0.327273
Round 30
0.353535
Round 40
0.393939
Round 50
0.414141
Round 60
0.448485
Round 70
0.478788
Round 80
0.490909
Round 90
0.49697
Round 100
0.515152
Round 110
0.519192
Round 120
0.535354
Round 130
0.543434
Round 140
0.553535
Round 150
0.565657
Round 160
0.573737
Round 170
0.567677
Round 180
0.577778
Round 190
0.583838


In [ ]:
for i in range(400):
    train_cycle()
    if i % 10 == 0:
        print("Round {}".format(200 + i))
        test_cycle()

Round 200
0.593939
Round 210
0.581818
Round 220
0.59596
Round 230
0.60404
Round 240
0.60404
Round 250
0.59596
Round 260
0.60202
Round 270
0.589899
Round 280
0.606061
Round 290
0.614141
Round 300
0.610101
Round 310
0.628283
Round 320
0.612121
Round 330
0.616162
Round 340
0.606061
Round 350
0.628283
Round 360
0.616162
Round 370
0.616162
Round 380
0.624242
Round 390
0.622222
Round 400
0.632323
Round 410
0.622222
Round 420
0.630303
Round 430
0.616162
Round 440
0.620202
Round 450
0.638384
Round 460
0.638384
Round 470
0.626263
Round 480
0.636364
Round 490
0.612121
Round 500
0.644444
Round 510
0.636364
Round 520
0.632323
Round 530
0.638384
Round 540
0.638384
Round 550
0.658586
Round 560
0.636364
Round 570
0.634343
